In [2]:
import pandas as pd
import numpy as np
import zipfile
import psycopg2 
import json
import os


In [15]:
# create file path variable
file_path = "Downloads/Churn_Modeling_v1.zip"


# open zipped dataset
with zipfile.ZipFile(file_path) as z:
   # open the csv file in the dataset
   with z.open("Churn_Modeling_v1.csv") as f:
       
      # read the dataset
      df = pd.read_csv(f)
       
# display dataset
df.head()

,id,last_name,credit_score,location,gender,age,tenure,balance,product_count,card_count,member_active,est_salary,exited,risk_score,sales_op,high_bal,hash
0,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1,NaN,NaN,NaN,f8ab46c7-2486-49a3-93f7-744db22384f1
1,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0,NaN,NaN,NaN,f8ab46c7-2486-49a3-93f7-744db22384f1
2,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1,NaN,NaN,NaN,f8ab46c7-2486-49a3-93f7-744db22384f1
3,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0,NaN,NaN,NaN,f8ab46c7-2486-49a3-93f7-744db22384f1
4,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0,NaN,NaN,NaN,f8ab46c7-2486-49a3-93f7-744db22384f1


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 17 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             10000 non-null  int64  
 1   last_name      10000 non-null  object 
 2   credit_score   10000 non-null  int64  
 3   location       10000 non-null  object 
 4   gender         10000 non-null  object 
 5   age            10000 non-null  int64  
 6   tenure         10000 non-null  int64  
 7   balance        10000 non-null  float64
 8   product_count  10000 non-null  int64  
 9   card_count     10000 non-null  int64  
 10  member_active  10000 non-null  int64  
 11  est_salary     10000 non-null  float64
 12  exited         10000 non-null  int64  
 13  risk_score     0 non-null      float64
 14  sales_op       0 non-null      float64
 15  high_bal       0 non-null      float64
 16  hash           10000 non-null  object 
dtypes: float64(5), int64(8), object(4)
memory usage: 1.

In [16]:
# Risk score condition
df['risk_score'] = df['balance'] / df['est_salary']

# find balances over 50,000 
df['high_bal'] = df['balance'] > 50000

# base level sale opportunity indication
df['sales_op'] = df.apply(lambda row: 1 if row['card_count'] == 0 else 0, axis=1)


df.head(10)

,id,last_name,credit_score,location,gender,age,tenure,balance,product_count,card_count,member_active,est_salary,exited,risk_score,sales_op,high_bal,hash
0,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1,0.000000,0,False,f8ab46c7-2486-49a3-93f7-744db22384f1
1,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0,0.744677,1,True,f8ab46c7-2486-49a3-93f7-744db22384f1
2,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1,1.401375,0,True,f8ab46c7-2486-49a3-93f7-744db22384f1
3,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0,0.000000,1,False,f8ab46c7-2486-49a3-93f7-744db22384f1
4,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0,1.587055,0,True,f8ab46c7-2486-49a3-93f7-744db22384f1
5,15574012,Chu,645,Spain,Male,44,8,113755.78,2,1,0,149756.71,1,0.759604,0,True,f8ab46c7-2486-49a3-93f7-744db22384f1
6,15592531,Bartlett,822,France,Male,50,7,0.00,2,1,1,10062.80,0,0.000000,0,False,f8ab46c7-2486-49a3-93f7-744db22384f1
7,15656148,Obinna,376,Germany,Female,29,4,115046.74,4,1,0,119346.88,1,0.963969,0,True,f8ab46c7-2486-49a3-93f7-744db22384f1
8,15792365,He,501,France,Male,44,4,142051.07,2,0,1,74940.50,0,1.895518,1,True,f8ab46c7-2486-49a3-93f7-744db22384f1
9,15592389,H?,684,France,Male,27,2,134603.88,1,1,1,71725.73,0,1.876647,0,True,f8ab46c7-2486-49a3-93f7-744db22384f1


In [3]:
# Read the credentials from a json file
with open("Downloads/credentials.json", "r") as f:
    admin_credentials = json.load(f)
# Format the connection parameters as a string
credentials = "host={host} dbname={dbname} user={user} password={password} port={port} ".format(**admin_credentials)

# Connect to the database using the formatted string
conn = psycopg2.connect(credentials)

# Close the connection after use
conn.close()

In [11]:
# connecting to target database
def connect_to_db(credentials, dbname):
    # Read the credentials from a json file
    with open("C:/Users/William/Downloads/credentials.json", "r") as f:
        creds = json.load(f)

    # Format the connection parameters as a string
    conn_string = "host={host} port={port} dbname={dbname} user={user} password={password}".format(
        host=creds["host"],
        port=creds["port"],
        dbname=dbname,
        user=creds["user"],
        password=creds["password"]
    )

    # Connect to the database using the formatted string
    conn = psycopg2.connect(conn_string)

# Close the connection after use
conn.close()

In [5]:
# Show all databases

def show_databases():
    # Connect to the PostgreSQL database server
    conn = psycopg2.connect(credentials)

    # Create a cursor object
    cursor = conn.cursor()

    # Execute the SQL statement to retrieve all databases
    cursor.execute("SELECT datname FROM pg_database")

    # Fetch all the results
    databases = cursor.fetchall()

    # Print the list of databases
    print("List of databases:")
    for database in databases:
        print("-", database[0])

    # Close the cursor and connection
    cursor.close()
    conn.close()

# Call the function to display all databases
show_databases()


List of databases:
- postgres
- mysite_db
- template1
- template0
- BANK_DATABASE
- test
- customers


In [6]:
# check for existence of database in postgres
def check_database_exists(database_name):
    # Connect to the PostgreSQL database server
    conn = psycopg2.connect(credentials)

    # Create a cursor object
    cursor = conn.cursor()

    # Execute the SQL statement to retrieve all databases
    cursor.execute("SELECT datname FROM pg_database")

    # Fetch all the results
    databases = cursor.fetchall()

    # Check if the specified database exists
    exists = False
    for db in databases:
        if db[0] == database_name:
            exists = True
            break

    # Close the cursor and connection
    cursor.close()
    conn.close()

    return exists

# Call the function to check for the existence of a database
database_name = "test_db"
if check_database_exists(database_name):
    print(f"The database '{database_name}' exists.")
else:
    print(f"The database '{database_name}' does not exist.")

check_database_exists("customers")

The database 'test_db' does not exist.


True

In [7]:
# create database in postgres

def create_database(database_name):
    # Connect to the PostgreSQL database server
    conn = psycopg2.connect(credentials)

    # Create a cursor object
    cursor = conn.cursor()

    # use autocommit to create database or else you will get error ActiveSqlTransaction: CREATE DATABASE cannot run inside a transaction block
    conn.autocommit = True #!

    # Close any active transaction
    conn.commit()

    # Execute the SQL statement to create the new database
    cursor.execute(f"CREATE DATABASE {database_name}")

    # Commit the transaction
    conn.commit()

    # Close the cursor and connection
    cursor.close()
    conn.close()

# Call the function to create a new database
database_name = "subcribers"
create_database(database_name)

print(f"The database '{database_name}' has been created.")


The database 'subcribers' has been created.


In [14]:
# create table in postgres

def create_table(conn, table_name):
    # Create a cursor object
    cursor = conn.cursor()

    # Execute the SQL statement to create the new table
    cursor.execute(f"""
        CREATE TABLE IF NOT EXISTS {table_name} (
            id INTEGER,
            last_name VARCHAR(255),
            credit_score INTEGER,
            location VARCHAR(255),
            gender VARCHAR(255),
            age INTEGER,
            tenure INTEGER,
            balance FLOAT,
            product_count INTEGER,
            card_count INTEGER,
            member_active BOOLEAN,
            est_salary FLOAT,
            exited BOOLEAN,
            risk_score FLOAT,
            sales_op INTEGER,
            high_bal BOOLEAN,
            hash VARCHAR(255)
        );
    """)

    # Commit the transaction
    conn.commit()

    # Close the cursor
    cursor.close()

# Connect to the PostgreSQL database server
conn = psycopg2.connect(credentials, dbname="subcribers")

# Call the function to create the new table
table_name = "customer"
create_table(conn, table_name)
print(f"The table '{table_name}' has been created.")

# Close the connection
conn.close()


The table 'customer' has been created.


In [17]:
from io import StringIO

def copy_to_table(conn, df, table_name):
    # Get the column names for the table
    cursor = conn.cursor()
    cursor.execute(f"SELECT column_name FROM information_schema.columns WHERE table_name = '{table_name}'")
    table_columns = [row[0] for row in cursor.fetchall()]
    cursor.close()

    # Filter the DataFrame to only include the columns that are in the table
    df = df[table_columns]

    # Convert the DataFrame to a CSV-formatted string
    s = StringIO()
    df.to_csv(s, index=False, header=False)
    s.seek(0)

    # Use the COPY command to copy the data into the table
    cursor = conn.cursor()
    cursor.copy_from(s, table_name, columns=table_columns, sep=",")
    conn.commit()
    cursor.close()

conn = psycopg2.connect(credentials, dbname="subcribers")

copy_to_table(conn, df,"customer")


In [18]:
# Copy from main customer table to sales_opps table
conn = psycopg2.connect(credentials, dbname="subcribers")

cursor = conn.cursor()

cursor.execute("""
CREATE TABLE sales_opps AS 
SELECT
    id,
    last_name,
    location,
    gender,
    age,
    balance,
    product_count,
    card_count,
    member_active,
    est_salary,
    exited,
    risk_score,
    sales_op,
    high_bal,
    hash
FROM
    customer
WHERE 
    sales_op = 1
""")

conn.commit()
cursor.close()
conn.close()